# Chapter 11-02: CUDA C++ 커널 작성과 Python 연동

## 학습 목표
- **CUDA C++** 커널 문법(`__global__`, `blockIdx`, `threadIdx`, `blockDim`)을 이해하고 벡터/행렬 연산 커널을 작성한다
- **Shared Memory Tiling**으로 Global Memory 접근을 최소화하는 행렬 곱 커널을 구현한다
- **pybind11** 또는 TensorFlow `tf.custom_op` / **CuPy**로 커스텀 커널을 Python에 노출하는 방법을 학습한다
- Naive CUDA GEMM vs cuBLAS vs Tiled CUDA의 성능 차이를 Roofline 관점에서 분석한다

## 목차
1. [수학적 기초: GEMM의 Shared Memory Tiling](#1.-수학적-기초)
2. [CUDA 커널 구조 이해 (의사코드 분석)](#2.-CUDA-커널-구조)
3. [Python 레벨 GPU 연산 성능 분석](#3.-Python-GPU-성능)
4. [Tiled GEMM 시뮬레이션 (NumPy)](#4.-Tiled-GEMM-시뮬레이션)
5. [메모리 접근 패턴 비교 시각화](#5.-메모리-접근-패턴)
6. [정리 및 연습 문제](#6.-정리)

## 1. 수학적 기초

### Tiled GEMM: Shared Memory 활용

표준 GEMM $C = A \times B$ ($A: M\times K$, $B: K\times N$, $C: M\times N$)에서,  
타일 크기 $T$로 분할하면:

$$C_{ij} = \sum_{k=0}^{K-1} A_{ik} B_{kj} = \sum_{t=0}^{K/T - 1} \sum_{\ell=0}^{T-1} A_{i, tT+\ell} \cdot B_{tT+\ell, j}$$

각 타일 $t$에서:
1. $A$의 $M \times T$ 서브행렬을 Shared Memory에 로드
2. $B$의 $T \times N$ 서브행렬을 Shared Memory에 로드
3. 두 서브행렬의 곱을 레지스터에 누적
4. 다음 타일로 이동 (`__syncthreads()`로 동기화)

### 메모리 접근 절약 분석

- **Naive**: 각 $C_{ij}$ 연산 시 $A$의 K개 + $B$의 K개 원소를 Global Memory에서 직접 읽음
  → 총 Global Memory 읽기: $MNK + MNK = 2MNK$ elements
- **Tiled**: 타일 1개를 Shared Memory에 1번 로드 후 $T$번 재사용
  → 총 Global Memory 읽기: $\frac{2MNK}{T}$ elements → **$T$배 절약**!

---

### 🐣 초등학생을 위한 친절 설명!

#### 📦 Shared Memory Tiling이 뭔가요?

> 도서관(Global Memory)에서 책을 빌릴 때마다 먼 곳까지 가야 해요 (느림!).  
> **해결책**: 책 여러 권을 한 번에 빌려서 책상(Shared Memory) 위에 쌓아두고,  
> 여러 계산에 재사용한다면? → 도서관 방문 횟수를 T배 줄일 수 있어요!

### 📝 연습 문제

#### 문제 1: Tiled GEMM 메모리 절약 계산

$M=K=N=4096$, 타일 크기 $T=32$일 때  
Naive vs Tiled GEMM의 Global Memory 읽기 횟수를 비교하라.

<details>
<summary>💡 풀이 확인</summary>

$M = K = N = 4096$, $T = 32$

- **Naive**: $2MNK = 2 \times 4096^3 \approx 137.4 \text{ G elements}$
- **Tiled**: $2MNK/T = 137.4 / 32 \approx 4.3 \text{ G elements}$  
- 절약량: **32배** (타일 크기와 동일!)

FP16 기준 바이트: Naive $\approx 275$ GB, Tiled $\approx 8.6$ GB
</details>

#### 문제 2: 이상적 AI (Tiled GEMM)

$M=K=N=4096$, FP16 Tiled GEMM(T=32)의 FLOPs와 메모리 접근량으로 AI를 계산하고  
H100 Ridge Point(295 FLOPs/Byte)와 비교하라.

<details>
<summary>💡 풀이 확인</summary>

FLOPs = $2 \times 4096^3 \approx 137.4$ GFLOPS  
Bytes = $2MNK/T \times 2$ bytes (FP16) ≈ $17.2$ GB  
AI = $137.4/17.2 \approx \mathbf{8000}$ FLOPs/Byte

AI (8000) ≫ Ridge (295) → **강한 Compute-bound** → cuBLAS와 맞먹는 효율 달성 가능!
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

np.random.seed(42)

## 2. CUDA 커널 구조

아래는 CUDA C++로 작성한 Tiled GEMM 커널의 의사코드입니다.  
실제 GPU 없이도 알고리즘 로직을 완전히 이해할 수 있습니다.

In [ ]:
# ---------------------------------------------------
# CUDA Tiled GEMM 커널 의사코드 설명
# (실제 CUDA C++ 코드를 주석으로 상세히 설명)
# ---------------------------------------------------

cuda_pseudocode = '''
/* ─────────────────────────────────────────────────────────────
   CUDA Tiled GEMM Kernel  (C = A × B)
   타일 크기: TILE_SIZE x TILE_SIZE
───────────────────────────────────────────────────────────── */

__global__ void tiled_gemm(float* A, float* B, float* C,
                            int M, int K, int N) {

    // ① 각 스레드의 전역 행/열 좌표 계산
    int row = blockIdx.y * blockDim.y + threadIdx.y;  // C의 행 좌표
    int col = blockIdx.x * blockDim.x + threadIdx.x;  // C의 열 좌표

    // ② Shared Memory 선언 (타일 2개)
    __shared__ float tileA[TILE_SIZE][TILE_SIZE];  // A의 서브행렬
    __shared__ float tileB[TILE_SIZE][TILE_SIZE];  // B의 서브행렬

    float result = 0.0f;  // 레지스터에 부분 합산 누적

    // ③ 타일 단위 루프 (K 차원을 TILE_SIZE씩 처리)
    for (int tile = 0; tile < K / TILE_SIZE; tile++) {

        // ④ 각 스레드가 A, B의 원소 하나씩 Shared Memory에 로드
        tileA[threadIdx.y][threadIdx.x] = A[row * K + tile * TILE_SIZE + threadIdx.x];
        tileB[threadIdx.y][threadIdx.x] = B[(tile * TILE_SIZE + threadIdx.y) * N + col];

        __syncthreads();  // ⑤ 모든 스레드 로드 완료 대기

        // ⑥ 로드된 타일로 내적 계산 (Global Memory 접근 없음!)
        for (int k = 0; k < TILE_SIZE; k++)
            result += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];

        __syncthreads();  // ⑦ 다음 타일 로드 전 동기화
    }

    // ⑧ 최종 결과를 Global Memory에 쓰기 (1회)
    if (row < M && col < N)
        C[row * N + col] = result;
}
'''

print(cuda_pseudocode)

print("\n주요 CUDA 개념 정리:")
print("  __global__        : GPU 커널 함수 (Host에서 호출, Device에서 실행)")
print("  blockIdx.x/y      : 이 Block의 Grid 내 좌표")
print("  threadIdx.x/y     : 이 Thread의 Block 내 좌표")
print("  __shared__        : Block 내 스레드가 공유하는 고속 Shared Memory")
print("  __syncthreads()   : Block 내 모든 스레드가 이 줄에 도달할 때까지 대기")
print("  ⑥ Global Memory 접근 = 0 → Tiling의 핵심!")

## 3. Python GPU 성능

In [ ]:
# ---------------------------------------------------
# TF/NumPy로 GEMM 성능 측정 및 이론 대비 효율 분석
# ---------------------------------------------------

import tensorflow as tf

def benchmark_gemm(M, K, N, n_warmup=5, n_runs=20):
    """행렬 곱 성능 측정 (GFLOPS, 이론 peak 대비 효율)"""
    A = tf.random.normal([M, K], dtype=tf.float32)
    B = tf.random.normal([K, N], dtype=tf.float32)

    # Warmup
    for _ in range(n_warmup):
        _ = tf.matmul(A, B)

    # 측정
    start = time.perf_counter()
    for _ in range(n_runs):
        C = tf.matmul(A, B)
    # CPU 동기화
    _ = C.numpy()
    elapsed = (time.perf_counter() - start) / n_runs

    flops = 2 * M * K * N  # 곱셈 + 덧셈
    gflops = flops / elapsed / 1e9
    return elapsed * 1000, gflops  # ms, GFLOPS

sizes = [
    (64, 4096, 4096),
    (256, 4096, 4096),
    (1024, 4096, 4096),
    (4096, 4096, 4096),
]

print(f"{'M':>6} | {'K':>6} | {'N':>6} | {'시간(ms)':>10} | {'GFLOPS':>10} | {'FP32 이론 대비'}")  
print("-" * 65)

peak_fp32_gflops = 989 * 1000  # H100 FP32 ≈ 989 TFLOPS → 989,000 GFLOPS
# Apple M3 (테스트 환경)은 약 14 TFLOPS
peak_estimate = 14 * 1000

results = []
for M, K, N in sizes:
    try:
        ms, gflops = benchmark_gemm(M, K, N)
        efficiency = gflops / peak_estimate * 100
        print(f"{M:>6} | {K:>6} | {N:>6} | {ms:>10.2f} | {gflops:>10.1f} | {efficiency:.1f}%")
        results.append((M, K, N, ms, gflops))
    except Exception as e:
        print(f"{M:>6} | ... (오류: {str(e)[:40]})")

print("\n→ 행렬이 클수록 GFLOPS 효율 증가 (Compute-bound 구간에 진입)")
print("→ 작은 배치(M=64): Memory-bound → 효율 낮음")

## 4. Tiled GEMM 시뮬레이션

In [ ]:
# ---------------------------------------------------
# Tiled GEMM을 순수 NumPy로 구현하고 정확도 검증
# (CUDA Shared Memory Tiling 알고리즘의 Python 구현)
# ---------------------------------------------------

def tiled_gemm_numpy(A, B, tile_size=4):
    """
    CUDA Tiled GEMM의 Python/NumPy 구현.
    CUDA의 Shared Memory를 Python 배열로 시뮬레이션.
    
    이 구현은 CUDA 커널의 논리를 1:1로 매핑:
      - outer loop = 타일 인덱스 (blockIdx처럼)
      - inner loop = 각 타일 내 연산 (threadIdx처럼)
    """
    M, K = A.shape
    K2, N = B.shape
    assert K == K2, "행렬 차원 불일치"
    C = np.zeros((M, N), dtype=A.dtype)

    global_mem_reads = 0
    shared_mem_reads = 0

    for tile_k in range(0, K, tile_size):
        t_end = min(tile_k + tile_size, K)

        for row_start in range(0, M, tile_size):
            r_end = min(row_start + tile_size, M)

            for col_start in range(0, N, tile_size):
                c_end = min(col_start + tile_size, N)

                # ── Shared Memory 로드 (Global Memory 접근) ──
                tileA = A[row_start:r_end, tile_k:t_end]      # Shared Mem에 로드
                tileB = B[tile_k:t_end, col_start:c_end]      # Shared Mem에 로드
                global_mem_reads += tileA.size + tileB.size

                # ── 타일 내 연산 (Shared Memory에서 읽기) ──
                C[row_start:r_end, col_start:c_end] += tileA @ tileB
                shared_mem_reads += tileA.size + tileB.size  # 재사용 횟수 추적

    return C, global_mem_reads, shared_mem_reads


# 검증
M, K, N = 8, 8, 8
A = np.random.randn(M, K).astype(np.float32)
B = np.random.randn(K, N).astype(np.float32)

C_ref   = A @ B   # 표준 GEMM
C_tiled, glb_reads, shr_reads = tiled_gemm_numpy(A, B, tile_size=4)

err = np.abs(C_ref - C_tiled).max()
print(f"[Tiled GEMM 검증] M={M}, K={K}, N={N}, tile=4")
print(f"  최대 오차: {err:.2e}")
print(f"  정확도:    {'✅ 완전 일치' if err < 1e-5 else '❌ 불일치'}")
print(f"\nGlobal Memory 읽기: {glb_reads} elements (Naive = {2*M*K*N//4})")  # 근사
print("→ 타일링으로 Global Memory 접근 횟수를 크게 줄임")

# 더 큰 행렬로 성능 비교
M2 = K2 = N2 = 64
A2 = np.random.randn(M2, K2).astype(np.float32)
B2 = np.random.randn(K2, N2).astype(np.float32)

t0 = time.perf_counter()
C_ref2 = A2 @ B2
t_std = time.perf_counter() - t0

t0 = time.perf_counter()
C_tiled2, _, _ = tiled_gemm_numpy(A2, B2, tile_size=8)
t_tiled = time.perf_counter() - t0

err2 = np.abs(C_ref2 - C_tiled2).max()
print(f"\n[{M2}×{K2}×{N2} GEMM 정확도]: 최대 오차={err2:.2e} {'✅' if err2 < 1e-4 else '❌'}")
print(f"(참고: Python 구현은 CUDA 병렬화 없어 느림. 실제 CUDA에서는 압도적으로 빠름)")

## 5. 메모리 접근 패턴

In [ ]:
# ---------------------------------------------------
# 타일 크기에 따른 Global Memory 접근 횟수 비교
# ---------------------------------------------------

M = K = N = 4096
tile_sizes = [1, 2, 4, 8, 16, 32, 64]

naive_reads = 2 * M * K * N  # Naive: 2MNK 원소 읽기
tiled_reads = [2 * M * K * N / T for T in tile_sizes]
savings_pct  = [(1 - t / naive_reads) * 100 for t in tiled_reads]

print(f"Naive GEMM Global Memory 읽기: {naive_reads/1e9:.1f}G elements ({naive_reads*2/1e9:.0f}GB FP16)")
print(f"\n{'타일 크기 T':>12} | {'읽기 횟수':>14} | {'절약률':>8}")
print("-" * 40)
for T, r, s in zip(tile_sizes, tiled_reads, savings_pct):
    print(f"{T:>12} | {r/1e9:>12.2f}G | {s:>7.1f}%")

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# 왼쪽: Global Memory 읽기 횟수
ax1 = axes[0]
ax1.semilogy(tile_sizes, [naive_reads/1e9]*len(tile_sizes), 'r--', lw=2.5, label='Naive')
ax1.semilogy(tile_sizes, [r/1e9 for r in tiled_reads], 'b-o', lw=2.5, ms=8, label='Tiled')
for T, r in zip(tile_sizes, tiled_reads):
    ax1.text(T, r/1e9*1.3, f'{r/1e9:.1f}G', ha='center', fontsize=8, color='blue')
ax1.set_xlabel('타일 크기 T', fontsize=11)
ax1.set_ylabel('Global Memory 읽기 (G elements, log)', fontsize=10)
ax1.set_title(f'Tiled GEMM: 타일 크기별\nGlobal Memory 접근 감소 (M=K=N={M})', fontweight='bold')
ax1.legend(fontsize=10); ax1.grid(True, alpha=0.3)

# 오른쪽: Naive vs Tiled 메모리 vs 연산 비교 (Roofline 개념)
ax2 = axes[1]
methods = ['Naive\nT=1', 'Tiled\nT=4', 'Tiled\nT=16', 'Tiled\nT=32', 'Ideal\n(cuBLAS)']
ai_vals  = [1.0, 4.0, 16.0, 32.0, 65.0]   # 정규화된 AI
perf_pct = [2.0, 12.0, 45.0, 80.0, 95.0]  # 이론 Peak 달성률 (추정)
colors_m = ['#E53935', '#FB8C00', '#FDD835', '#43A047', '#1E88E5']

bars = ax2.bar(methods, perf_pct, color=colors_m, alpha=0.85, edgecolor='white', lw=2)
for bar, pct in zip(bars, perf_pct):
    ax2.text(bar.get_x() + bar.get_width()/2, pct + 1.5,
             f'{pct}%', ha='center', fontsize=10, fontweight='bold')

ax2.axhline(y=100, color='k', ls='--', lw=1.5, alpha=0.5, label='이론 Peak 100%')
ax2.set_ylabel('이론 Peak FLOPS 달성률 (%)', fontsize=11)
ax2.set_title('구현 방식별 GPU 연산 효율\n(커스텀 커널 → cuBLAS 수준 목표)', fontweight='bold')
ax2.set_ylim(0, 115)
ax2.legend(fontsize=9); ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. 정리

### 핵심 개념 요약

| 개념 | 설명 |
|------|------|
| **`__global__`** | GPU에서 실행될 커널 함수 선언 |
| **`__shared__`** | Block 내 스레드 공유 고속 메모리 |
| **`__syncthreads()`** | Block 내 스레드 배리어 동기화 |
| **Tiling** | Shared Memory를 이용해 Global Memory 접근 $T$배 절약 |
| **Coalesced Access** | 인접 스레드가 연속 메모리 접근 → 대역폭 최대화 |
| **AI 향상** | Tiling으로 AI ≈ $T\times$ 증가 → Compute-bound로 전환 |

### 핵심 수식

$$\text{Global Memory 절감} = T \text{배}, \quad \text{AI}_{tiled} \approx T \times \text{AI}_{naive}$$

### 다음 챕터 예고
**Chapter 11-03: Triton 커널 프로그래밍** — C++ 없이 Python 문법으로 GPU 커널을 작성하는 Triton. Block Pointer, 자동 타일링, OpenAI Triton으로 Fused Softmax 등의 커스텀 연산을 구현한다.